# Módulo Grupo 2: Análise de Filtros de Suavização

**Objetivo:** Converter imagens para escala de cinza, aplicar filtros de suavização (Gaussiano, Bilateral, Mediano) e avaliar qual filtro é o mais eficaz, medindo a redução de ruído versus a perda de foco (nitidez).

# Célula 1: Importação das bibliotecas necessárias

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
from IPython.display import display, clear_output

print("Bibliotecas importadas com sucesso")

# Célula 2: Definição das Funções de Conversão, Filtros e Métricas

In [ ]:
def converter_para_cinza(img_bgr):
    """Converte uma imagem BGR para escala de cinza."""
    return cv.cvtColor(img_bgr, cv.COLOR_BGR2GRAY)

def medir_ruido(img):
    """Calcula o nível de ruído como o desvio padrão dos pixels."""
    return np.std(img)

def medir_foco_laplaciano(img_cinza):
    """Avalia a nitidez pela variância do Laplaciano.
    Valores altos indicam maior nitidez."""
    
    return cv.Laplacian(img_cinza, cv.CV_64F).var()

def aplicar_filtros(img_cinza):
    """Aplica os filtros: Gaussiano, Mediano e Bilateral."""
    img_gaussiano = cv.GaussianBlur(img_cinza, (5, 5), 0)
    img_mediano = cv.medianBlur(img_cinza, 5)
    img_bilateral = cv.bilateralFilter(img_cinza, 9, 75, 75)
    
    filtros = {
        'Gaussiano': img_gaussiano,
        'Mediano': img_mediano,
        'Bilateral': img_bilateral
    }
    return filtros

imagem_selecionada_bgr = None

print("Funções de análise definidas com sucesso")

# Célula 3: Upload de Imagem

In [ ]:
imagem_selecionada_bgr = None

print("Por favor, selecione uma imagem (JPG, JPEG, PNG).")
uploaded = files.upload()

if uploaded:
    nome_arquivo = list(uploaded.keys())[0]
    extensao = nome_arquivo.split('.')[-1].lower()
    
    if extensao in ['jpg', 'jpeg', 'png']:
        try:
            buffer_imagem = np.frombuffer(uploaded[nome_arquivo], np.uint8)
            imagem_selecionada_bgr = cv.imdecode(buffer_imagem, cv.IMREAD_COLOR)
            print(f"\nImagem '{nome_arquivo}' carregada com sucesso!")
        except Exception as e:
            print(f"\nErro ao processar '{nome_arquivo}': Arquivo corrompido ou inválido.")
    else:
        print(f"\nErro: Arquivo '{nome_arquivo}' IGNORADO. Formato não suportado.")
else:
    print("\nNenhuma imagem foi selecionada.")

# Célula 4: Análise Completa com Métricas

In [ ]:
if imagem_selecionada_bgr is not None:
    print("Iniciando análise...")
    print("-" * 40)
    
    img_cinza_original = converter_para_cinza(imagem_selecionada_bgr)
    ruido_original = medir_ruido(img_cinza_original)
    foco_original = medir_foco_laplaciano(img_cinza_original)
    
    resultados = {
        'Análise': ['Original'],
        'Ruído': [ruido_original],
        'Foco': [foco_original]
    }
    
    imagens_filtradas = aplicar_filtros(img_cinza_original)
    print("Filtros aplicados...")
    
    for nome_filtro, img_filtrada in imagens_filtradas.items():
        ruido_filtro = medir_ruido(img_filtrada)
        foco_filtro = medir_foco_laplaciano(img_filtrada)
        
        resultados['Análise'].append(nome_filtro)
        resultados['Ruído'].append(ruido_filtro)
        resultados['Foco'].append(foco_filtro)
    
    df_resultados = pd.DataFrame(resultados)
    
    print("Análise concluída.")
    print("-" * 40)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    fig.suptitle("Comparação de Filtros com Métricas", fontsize=16)
    
    axes[0, 0].imshow(img_cinza_original, cmap='gray')
    axes[0, 0].set_title(f"Original (Cinza)\nRuído: {ruido_original:.2f} | Foco: {foco_original:.2f}")
    axes[0, 0].axis('off')
    
    dados_gauss = df_resultados[df_resultados['Análise'] == 'Gaussiano'].iloc[0]
    axes[0, 1].imshow(imagens_filtradas['Gaussiano'], cmap='gray')
    axes[0, 1].set_title(f"Filtro Gaussiano\nRuído: {dados_gauss['Ruído']:.2f} | Foco: {dados_gauss['Foco']:.2f}")
    axes[0, 1].axis('off')
    
    dados_median = df_resultados[df_resultados['Análise'] == 'Mediano'].iloc[0]
    axes[1, 0].imshow(imagens_filtradas['Mediano'], cmap='gray')
    axes[1, 0].set_title(f"Filtro Mediano\nRuído: {dados_median['Ruído']:.2f} | Foco: {dados_median['Foco']:.2f}")
    axes[1, 0].axis('off')
    
    dados_bilat = df_resultados[df_resultados['Análise'] == 'Bilateral'].iloc[0]
    axes[1, 1].imshow(imagens_filtradas['Bilateral'], cmap='gray')
    axes[1, 1].set_title(f"Filtro Bilateral\nRuído: {dados_bilat['Ruído']:.2f} | Foco: {dados_bilat['Foco']:.2f}")
    axes[1, 1].axis('off')
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()
    
    print("\n" + "="*60)
    print("Tabela de Métricas")
    print("="*60)
    display(df_resultados.set_index('Análise'))
    
else:
    print("Nenhuma imagem carregada. Execute a Célula 3 primeiro.")